# MESQUAL 202: KPI Collections and Pretty Tables

This notebook demonstrates how to extract, organize, and present KPI data in well-formatted tables with automatic unit normalization.

## Introduction

Building on the KPI framework fundamentals from notebook 201, this notebook focuses on practical data extraction and presentation:

1. **DataFrame Export**: Converting KPI collections to pandas DataFrames
2. **Unit Normalization**: Automatic unit selection for readable tables
3. **Comparison KPIs**: Creating and analyzing scenario differences
4. **Advanced Filtering**: Model property-based filtering
5. **Pretty Tables**: Publication-ready table formatting
6. **MultiIndex Support**: Hierarchical table organization

These techniques enable rapid creation of analysis-ready tables from complex multi-scenario KPI collections.

## Setup

First, we need to set up the environment. If you are on Colab, the first cell will clone and install all dependencies. You will have to restart the session afterwards and continue with cell 2. If you are in a local environment, make sure that you have followed the Getting started steps in the README, so that mesqual and all requirements are installed.

In [ ]:
import os

if "COLAB_RELEASE_TAG" in os.environ:
    import importlib.util

    def is_module_available(module_name):
        return importlib.util.find_spec(module_name) is not None

    if os.path.exists("mesqual-vanilla-studies") and is_module_available("mesqual"):
        print("✅ Environment already set up. Skipping installation.")
    else:
        print("🔧 Setting up Colab environment...")
        !git clone --recursive https://github.com/helgeesch/mesqual-vanilla-studies.git
        %cd mesqual-vanilla-studies/

        !pip install git+https://github.com/helgeesch/mesqual -U
        !pip install git+https://github.com/helgeesch/mesqual-pypsa -U
        !pip install git+https://github.com/helgeesch/captain-arro -U
        !pip install -r requirements.txt

        print('✅ Setup complete. 🔁 Restart the session, then skip this cell and continue with the next one.')
else:
    print("🖥️ Running locally. No setup needed.")

In [ ]:
import os

if "COLAB_RELEASE_TAG" in os.environ:
    import sys
    sys.path.append('/content/mesqual-vanilla-studies')
    os.chdir('/content/mesqual-vanilla-studies')
else:
    def setup_notebook_env():
        """Set working directory to repo root and ensure it's in sys.path."""
        import os
        import sys
        from pathlib import Path

        def find_repo_root(start_path: Path) -> Path:
            current = start_path.resolve()
            while current != current.parent:
                if (current / 'vanilla').exists():
                    return current
                current = current.parent
            raise FileNotFoundError(f"Repository root not found from: {start_path}")

        repo_root = find_repo_root(Path.cwd())
        os.chdir(repo_root)
        if str(repo_root) not in sys.path:
            sys.path.insert(0, str(repo_root))

    setup_notebook_env()

try:
    from mesqual import StudyManager
except ImportError:
    raise ImportError("❌ 'mesqual' not found. If you're running locally, make sure you've installed all dependencies as described in the README.")

if not os.path.isdir("studies"):
    raise RuntimeError(f"❌ 'studies' folder not found. Make sure your working directory is set to the mesqual-vanilla-studies root. Current working directory: {os.getcwd()}")

print("✅ Environment ready. Let's go!")

In [ ]:
import pandas as pd
from mesqual.kpis import FlagAggKPIBuilder, ComparisonKPIBuilder, Aggregations, ValueComparisons
from mesqual.units import Units
from vanilla.notebook_config import configure_clean_output_for_jupyter_notebook

configure_clean_output_for_jupyter_notebook()

# Pandas display options for better tables
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)
pd.set_option('display.width', None)

## Load Study and Generate KPIs

Let's load our study and create a comprehensive set of KPIs:

In [ ]:
from studies.study_01_intro_to_mesqual.scripts.setup_study_manager import get_scigrid_de_study_manager

study = get_scigrid_de_study_manager()

# Clear any existing KPIs
study.scen.clear_kpi_collection_for_all_child_datasets()
study.comp.clear_kpi_collection_for_all_child_datasets()

print("✅ Study loaded successfully")

In [ ]:
# Generate price KPIs for all scenarios
price_def = (
    FlagAggKPIBuilder()
    .for_flag('control_areas_t.vol_weighted_marginal_price')
    .with_aggregation(Aggregations.Mean)
    .for_all_objects()
    .build()
)

for dataset in study.scen.datasets:
    kpis = price_def.generate_kpis(dataset)
    dataset.kpi_collection.extend(kpis)
    
print(f"✅ Generated price KPIs for {len(study.scen.datasets)} scenarios")
print(f"   Total KPIs: {study.scen.get_merged_kpi_collection().size}")

## Part 1: Basic DataFrame Export

The simplest way to export KPIs is to convert them to a DataFrame:

In [ ]:
# Get KPI collection and export to DataFrame
kpi_collection = study.scen.get_merged_kpi_collection()

df = kpi_collection.to_dataframe()

print(f"DataFrame shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nFirst few rows:")
print(df.head())

## Part 2: Unit Handling Strategies

The KPI collection supports multiple unit handling strategies:

### Strategy 1: Original Units (Default)

In [ ]:
df_original = kpi_collection.to_dataframe(unit_handling='original')
print("Sample values with original units:")
print(df_original[['object_name', 'dataset_name', 'value', 'unit']].head())

### Strategy 2: Auto-Convert to Pretty Units

In [ ]:
df_pretty = kpi_collection.to_dataframe(unit_handling='auto_convert')
print("Sample values with auto-converted pretty units:")
print(df_pretty[['object_name', 'dataset_name', 'value', 'unit']].head())

### Strategy 3: Normalize to Collection

Find a single "pretty" unit that works well for the entire collection:

In [ ]:
df_normalized = kpi_collection.to_dataframe(normalize_to_collection=True)
print("All values normalized to common unit:")
print(df_normalized[['object_name', 'dataset_name', 'value', 'unit']].head())
print(f"\nAll units: {df_normalized['unit'].unique()}")

### Strategy 4: Target Unit

Convert all KPIs to a specific unit:

In [ ]:
df_eur_mwh = kpi_collection.to_dataframe(
    unit_handling='target',
    target_unit=Units.EUR / Units.MWh
)
print("All values in EUR/MWh:")
print(df_eur_mwh[['object_name', 'dataset_name', 'value', 'unit']].head())

## Part 3: Creating Pretty Tables

Let's create analysis-ready tables with proper organization:

### Pivot Table: Scenarios × Objects

In [ ]:
# Export with normalized units
df = kpi_collection.to_dataframe(normalize_to_collection=True)

# Create pivot table
pivot = df.pivot(index='object_name', columns='dataset_name', values='value')

# Get the common unit for display
common_unit = df['unit'].iloc[0]

print(f"Average Market Prices [{common_unit}]")
print("=" * 80)
print(pivot)

### Styled Table with Formatting

In [ ]:
# Create styled pivot table
styled = pivot.style\
    .format("{:.2f}")\
    .background_gradient(cmap='RdYlGn_r', axis=None)\
    .set_caption(f"Average Market Prices by Control Area and Scenario [{common_unit}]")

styled

## Part 4: Comparison KPIs

Create and analyze scenario comparison KPIs:

In [ ]:
# Generate comparison KPIs
for comp_dataset in study.comp.datasets:
    scenario_kpis = comp_dataset.variation_dataset.kpi_collection
    reference_kpis = comp_dataset.reference_dataset.kpi_collection
    
    for ref_kpi in reference_kpis:
        # Find matching scenario KPI
        matching = scenario_kpis.filter(
            flag=ref_kpi.attributes.flag,
            object_name=ref_kpi.attributes.object_name,
            aggregation=ref_kpi.attributes.aggregation
        )
        
        if matching.size > 0:
            scenario_kpi = matching[0]
            
            # Create comparison KPIs
            comp_def = (
                ComparisonKPIBuilder()
                .compare(scenario_kpi)
                .to(ref_kpi)
                .using_comparisons([
                    ValueComparisons.AbsoluteDifference,
                    ValueComparisons.RelativeDifference
                ])
                .build()
            )
            comp_kpis = comp_def.generate_kpis(None)
            comp_dataset.kpi_collection.extend(comp_kpis)

print(f"✅ Generated comparison KPIs for {len(study.comp.datasets)} comparisons")
print(f"   Total comparison KPIs: {study.comp.get_merged_kpi_collection().size}")

### Comparison Table: Absolute Differences

In [ ]:
# Get absolute difference KPIs
comp_kpis = study.comp.get_merged_kpi_collection()
abs_diff_kpis = comp_kpis.filter(value_comparison=ValueComparisons.AbsoluteDifference)

# Export and pivot
df_comp = abs_diff_kpis.to_dataframe(normalize_to_collection=True)
pivot_comp = df_comp.pivot(index='object_name', columns='dataset_name', values='value')

comp_unit = df_comp['unit'].iloc[0]

print(f"Price Changes vs Base Scenario [{comp_unit}]")
print("=" * 80)
print(pivot_comp)

### Relative Differences (Percentage Changes)

In [ ]:
# Get relative difference KPIs
rel_diff_kpis = comp_kpis.filter(value_comparison=ValueComparisons.RelativeDifference)

# Export and pivot
df_rel = rel_diff_kpis.to_dataframe()
pivot_rel = df_rel.pivot(index='object_name', columns='dataset_name', values='value')

# Convert to percentage
pivot_rel = pivot_rel * 100

print(f"Relative Price Changes vs Base Scenario [%]")
print("=" * 80)
print(pivot_rel)

## Part 5: Advanced Filtering

Use model properties to filter KPIs:

In [ ]:
# Get KPIs for TenneTDE control area only
all_kpis = study.scen.get_merged_kpi_collection()

tennet_kpis = all_kpis.filter_by_model_property(
    'object_name',
    value='TenneTDE'
)

print(f"KPIs for TenneTDE: {tennet_kpis.size}")

# Create table for single control area across scenarios
df_tennet = tennet_kpis.to_dataframe(normalize_to_collection=True)
print(f"\nTenneTDE Average Prices [{df_tennet['unit'].iloc[0]}]:")
print(df_tennet[['dataset_name', 'value']].set_index('dataset_name'))

## Part 6: Combined Analysis Tables

Combine scenarios and comparisons in unified tables:

In [ ]:
# Get both scenario and comparison KPIs
scenario_kpis = study.scen.get_merged_kpi_collection()
comparison_kpis = study.comp.get_merged_kpi_collection().filter(
    value_comparison=ValueComparisons.AbsoluteDifference
)

# Export both
df_scen = scenario_kpis.to_dataframe(normalize_to_collection=True)
df_comp = comparison_kpis.to_dataframe(normalize_to_collection=True)

# Add type indicator
df_scen['type'] = 'scenario'
df_comp['type'] = 'change'

# Combine
df_combined = pd.concat([df_scen, df_comp], ignore_index=True)

# Pivot for comparison
pivot_combined = df_combined.pivot(
    index='object_name',
    columns=['type', 'dataset_name'],
    values='value'
)

# Sort columns for better readability
pivot_combined = pivot_combined.sort_index(axis=1, level=[1, 0])

print("Combined Scenario Values and Changes")
print("=" * 120)
print(pivot_combined)

## Part 7: Summary Statistics Tables

In [ ]:
# Get scenario KPIs
df = scenario_kpis.to_dataframe(normalize_to_collection=True)

# Calculate summary statistics by scenario
summary = df.groupby('dataset_name')['value'].agg([
    ('min', 'min'),
    ('mean', 'mean'),
    ('max', 'max'),
    ('std', 'std')
])

unit = df['unit'].iloc[0]

print(f"Summary Statistics by Scenario [{unit}]")
print("=" * 80)
print(summary)

# Styled version
summary.style\
    .format("{:.2f}")\
    .background_gradient(cmap='YlOrRd', subset=['mean', 'max'])\
    .set_caption(f"Market Price Statistics by Scenario [{unit}]")

## Part 8: Exporting Tables

Export tables for reports and presentations:

In [ ]:
# Export to CSV
output_dir = 'non_versioned/output/kpi_tables'
import os
os.makedirs(output_dir, exist_ok=True)

# Scenario values
pivot.to_csv(f'{output_dir}/scenario_prices.csv')
print(f"✅ Exported scenario prices to {output_dir}/scenario_prices.csv")

# Comparison values
pivot_comp.to_csv(f'{output_dir}/price_changes.csv')
print(f"✅ Exported price changes to {output_dir}/price_changes.csv")

# Summary statistics
summary.to_csv(f'{output_dir}/summary_statistics.csv')
print(f"✅ Exported summary statistics to {output_dir}/summary_statistics.csv")

## Summary: KPI Collection and Table Capabilities

### DataFrame Export Strategies:

1. **Original Units** (`unit_handling='original'`)
   - Keep units as computed
   - Best for raw data export

2. **Auto-Convert** (`unit_handling='auto_convert'`)
   - Each KPI converted to its own pretty unit
   - Good for mixed KPI types

3. **Normalize to Collection** (`normalize_to_collection=True`)
   - Single common unit for entire collection
   - **Best for tables and comparisons**
   - Uses median order of magnitude

4. **Target Unit** (`unit_handling='target', target_unit=...`)
   - Explicit unit specification
   - Useful for standard reporting formats

### Table Creation Patterns:

- **Pivot Tables**: `.pivot(index='object_name', columns='dataset_name', values='value')`
- **Summary Stats**: `.groupby('dataset_name')['value'].agg([...])`
- **MultiIndex**: Hierarchical column organization
- **Styling**: `.style.format().background_gradient()`

### Comparison Workflows:

1. Generate scenario KPIs
2. Create comparison KPIs with `ComparisonKPIBuilder`
3. Filter by `value_comparison` type
4. Export to DataFrame with normalized units
5. Pivot and format

### Best Practices:

✅ Use `normalize_to_collection=True` for consistent tables
✅ Filter before export to reduce data size
✅ Add type indicators when combining different KPI types
✅ Use styling for visual emphasis
✅ Export to CSV for sharing and archival

## Conclusion

The MESQUAL KPI collection system provides:

✅ **Flexible Export** - Multiple unit handling strategies
✅ **Automatic Normalization** - Smart unit selection for readable tables
✅ **Rich Filtering** - By attributes and model properties
✅ **Easy Pivoting** - Transform to analysis-ready formats
✅ **Publication Quality** - Styled tables with proper formatting

These capabilities enable rapid creation of professional analysis tables from complex multi-scenario energy system studies, with minimal code and maximum clarity.